In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Loading dataset
data = pd.read_csv("C:\\UnderGrad\\data\\ChargePoint_Data_CY20Q4.csv")
data.head()

In [ ]:
data.rename(columns={'Energy (kWh)':'Energy'}, inplace=True)
data['Start Date'] = pd.to_datetime(data['Start Date'])
#Flooring the date to a daily format
data['Start Date'] = data['Start Date'].dt.floor('D')
data.set_index('Start Date', inplace=True)
data.head()

In [ ]:
data = data.loc['2012-01-01':]
data

### T-GCN Setup

In [ ]:
from math import sin,cos,sqrt,atan2,radians
import scipy.sparse as sp
import networkx as nx
def distance(x,y):
    '''
        Calculates the distance between two points x & y.
        Returns the distance in meters.
    '''
    R=6373.0 
    lat_1 = radians(abs(x['Latitude']))
    lon_1 = radians(abs(x['Longitude']))
    lat_2 = radians(abs(y['Latitude']))
    lon_2 = radians(abs(y['Longitude']))
    
    lon_dist = lon_2 - lon_1
    lat_dist = lat_2 - lat_1
    
    #Haversine formula
    z = sin(lat_dist/2)**2 + (cos(lat_1) * cos(lat_2) * sin(lon_dist/2)**2)
    c = atan2(sqrt(z), sqrt(1-z)) * 2
    
    #distance in km
    dist = R*c
    return dist

def adj_norm(adj):
    '''
        Returns sparse normalized adjacency matrix
    '''
    d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
    adjnorm = d.dot(adj).transpose().dot(d).tocsr()

    return adjnorm


def getGraph(data):
    G = nx.Graph()
    
    #Creating a graph for each station
    for station in data['Station Name'].unique():
        G.add_node(station)
        G.nodes[station]['Station Name'] = data[data['Station Name']==station]['Station Name'].iloc[0]
        G.nodes[station]['Latitude'] = data[data['Station Name']==station]['Latitude'].iloc[0]
        G.nodes[station]['Longitude'] = data[data['Station Name']==station]['Longitude'].iloc[0]
        G.nodes[station]['Position'] = (G.nodes[station]['Longitude'], G.nodes[station]['Latitude'])
        
    for x in G.nodes:
        for y in G.nodes:
            dist = distance(G.nodes[x], G.nodes[y])
            if (dist>2.5):
                continue
            G.add_edge(x,y)
            G[x][y]['weight'] = np.exp(-dist)
    
    adj = nx.adjacency_matrix(G)
    adjnorm = adj_norm(adj).todense()
    return G, adj


### Train Test split  

In [ ]:
from datetime import datetime, timedelta

def setupTGCN(data,forecast_h):
    if forecast_h not in [1,7, 30]:
        raise ValueError('Forecasting horizon must be 1, 7 or 30')

    numlags=30
    #getting graph and adj matrix
    G, adj = getGraph(data)
    #calculating number of days in the data
    num_h = int(((data.index.max()-data.index.min()).total_seconds()))
    num_h = int(num_h//(3600*24))
    #initializing timeseries array
    ts = np.zeros([len(G.nodes()), num_h+1])
    starttime = data.index.min()

    #looping through each day
    for i in range(num_h+1):
        window = starttime + timedelta(days=i)
        current = data[(data.index == window)]

        #populating the array for each graph node
        for j, n in enumerate(G.nodes()):
            temp = current[G.nodes[n]['Station Name']==current['Station Name']]
            ts[j,i] = np.sum(temp['Energy'])

    #replacing values equal to zero with random numbers
    ts[ts==0] = np.random.normal(np.zeros_like(ts[ts==0]), 5)

    #normalizing data
    max_ = np.max(ts, axis=1)[:,None]
    norm = ts/max_

    #initializing lagged matrix
    num_nodes = len(G.nodes())
    matrixlags = np.zeros((ts.shape[-1]-(numlags+forecast_h), ts.shape[0], numlags+forecast_h))
    rng=matrixlags.shape[0]
    if forecast_h in [1,7]:
        itrain = rng - 30
    else:
        itrain = rng - 120
    itest=rng
    for i in range(rng):
        matrixlags[i] = norm[:,i:i+numlags+forecast_h]

    #-------------Train/Test split--------------------------
    X_train = np.zeros((itrain,num_nodes, numlags))
    y_train = np.zeros((itrain,num_nodes, forecast_h))
    X_test = np.zeros((itest-itrain,num_nodes, numlags))
    y_test = np.zeros((itest-itrain,num_nodes, forecast_h))

    for i, n in enumerate(G.nodes):
        X_train[:,i,:] = matrixlags[:itrain,i,:numlags]
        y_train[:,i,:] = matrixlags[:itrain,i,numlags:]
        X_test[:,i,:] = matrixlags[itrain:itest,i,:numlags]
        y_test[:,i,:] = matrixlags[itrain:itest,i,numlags:]

    #denormalizing
    X_test_denorm = X_test * max_
    y_test_denorm = y_test * max_

    print(f'X_train shape: {X_train.shape}')
    print(f'y_train shape: {y_train.shape}')
    print(f'X_test shape: {X_test.shape}')
    print(f'y_test shape: {y_test_denorm.shape}')
    return G, adj, X_train, y_train, X_test, y_test_denorm, max_


### TGCN Model Setup

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models,Input, optimizers, regularizers
from tensorflow.keras.layers import Layer,Input, Reshape, Permute, LSTM, BatchNormalization, Dense, Lambda

class GraphConvLayer(layers.Layer):
    def __init__(self, filters, adj, reg_lambda):
        super(GraphConvLayer, self).__init__()
        self.filters=filters
        self.adj=tf.cast(adj.toarray(), tf.float32) #ensuring adjacency matrix is float32
        self.reg_lambda = reg_lambda
    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.filters),
                                     initializer='glorot_uniform',
                                     regularizer=regularizers.l2(self.reg_lambda),
                                     trainable=True)
    
    def call(self, inputs):
        #adding self loop to adj matrix (A+I)
        adj_selfloop = self.adj + tf.eye(tf.shape(self.adj)[0], dtype=tf.float32)
        #computing degree matrix D_e
        degree_matrix = tf.reduce_sum(adj_selfloop, axis=-1)
        #computing D_e^{-1/2}
        degree_matrix_sqrt = tf.linalg.diag(tf.pow(degree_matrix, -0.5))
        #computing normalized adjacency matrix Ab = D_e^{-1/2}*(A+I)*D_e^{-1/2}
        adjnorm = tf.matmul(tf.matmul(degree_matrix_sqrt, adj_selfloop), degree_matrix_sqrt)
        #Graph Conv Ab*X*W
        x = tf.matmul(adjnorm, inputs) #Ab*X
        x = tf.matmul(x, self.kernel) #Ab*X*W
        x = tf.nn.relu(x)
        return x
    

In [ ]:
def buildTGCN(input_shape, adj, forecast_h):
    reg_lambda = 0.02
    nodes = input_shape[0]
    inputs = Input(shape=input_shape)
    #1st Graph Conv layer | 16 filters
    x = GraphConvLayer(16,adj, reg_lambda=reg_lambda)(inputs)
    #2nd Graph Conv layer | 10 filters
    x = GraphConvLayer(10,adj, reg_lambda=reg_lambda)(x)
    #LSTM layer
    x = layers.LSTM(50, return_sequences=False, kernel_regularizer=regularizers.l2(reg_lambda))(x)
    #dense layer with forecast horizon for each station
    x = layers.Dense(forecast_h*nodes, kernel_regularizer=regularizers.l2(reg_lambda))(x)
    #reshaping to (batch_size, stations, forecast_h)
    out = layers.Reshape((nodes, forecast_h))(x)
    
    mdl = models.Model(inputs=inputs, outputs=out)
    mdl.compile(optimizer=optimizers.Adam(), loss='mae')
    return mdl

### One Day Forecasting

In [ ]:
forecast_h=1
G, adj, X_train1, y_train1, X_test1, y_test1, max_ = setupTGCN(data, 1)

In [ ]:
input_shape = X_train1.shape[1:]
input_shape

In [ ]:
mdl1 = buildTGCN(input_shape, adj, 1)
mdl1.summary()

In [ ]:
mdl1.fit(X_train1,y_train1, epochs=100)

In [ ]:
pred1=mdl1.predict(X_test1)
pred1 = pred1 * max_
totalpred1 = np.sum(pred1, axis=(1))
totalactual1 = np.sum(y_test1, axis=(1))

In [ ]:
rmse1 = np.sqrt(np.mean((totalpred1-totalactual1)**2))
print(f'RMSE for T-GCN in 1 forecasting days: {rmse1}')

### Seven Day Forecasting

In [ ]:
forecast_h=7
G, adj, X_train7, y_train7, X_test7, y_test7, max_ = setupTGCN(data, 7)

In [ ]:
input_shape = X_train7.shape[1:]
input_shape

In [ ]:
mdl7 = buildTGCN(input_shape, adj, 7)
mdl7.summary()

In [ ]:
mdl7.fit(X_train7, y_train7, epochs=100)

In [ ]:
pred7=mdl7.predict(X_test7)
pred7 = pred7 * max_
totalpred7 = np.sum(pred7, axis=(1))
totalactual7 = np.sum(y_test7, axis=(1))

In [ ]:
rmse7 = np.sqrt(np.mean((totalpred7-totalactual7)**2))
print(f'RMSE for T-GCN in 7 forecasting days: {rmse7}')

### 30 Day Forecast

In [ ]:
forecast_h=30
G, adj, X_train30, y_train30, X_test30, y_test30, max_ = setupTGCN(data, 30)

In [ ]:
input_shape = X_train30.shape[1:]
input_shape

In [ ]:
mdl30 = buildTGCN(input_shape, adj, 30)
mdl30.summary()

In [ ]:
mdl30.fit(X_train30,y_train30, epochs=100)

In [ ]:
pred30=mdl30.predict(X_test30)
pred30=pred30 * max_
totalpred30 = np.sum(pred30, axis=1)
totalactual30 = np.sum(y_test30, axis=1)

In [ ]:
rmse30 = np.sqrt(np.mean((totalpred30-totalactual30)**2))
print(f'RMSE for T-GCN in 30 forecasting days: {rmse30}')